# RQ2 – Public Expectations Towards Corporate Digital Responsibility (CDR)


```
RQ2: What expectations does the public hold towards CDR, and how do public expectations differ across Environmental, Social, and Governance (ESG) dimensions of CDR?
```

*Purpose*: Validate the 3-dimensional structure of CDR expectations (E, S, G). Compare mean levels of expectations across these dimensions.

In [10]:
# ================================================
# RQ2 — EFA + CFA Model Suite (3F / 2ND / 2F / 1F)
# + Reliability + Codebook-Anchored Summary
# Pretty notebook printing (no Styler/Jinja)
# Research Drive I/O via rd_io
# ================================================
import warnings, sys, subprocess, re, io, posixpath
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from IPython.display import Markdown, display

# ---------- Research Drive I/O ----------
from rd_io import (
    OUT_DIR, rd_join,
    rd_read_parquet_df, rd_write_csv_df, rd_write_markdown, rd_upload_bytes,
    rd_read_text, rd_read_csv_df
)

# ---------- Install/load modeling deps ----------
def _ensure_semopy():
    try:
        from semopy import Model as SemopyModel  # noqa
        from semopy.stats import (  # noqa
            get_baseline_model, calc_dof, calc_chi2, calc_cfi, calc_tli, calc_rmsea, calc_aic, calc_bic
        )
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "semopy>=2.3.0", "--quiet"])

def _ensure_factor_analyzer():
    try:
        from factor_analyzer import FactorAnalyzer  # noqa
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "factor_analyzer>=0.5.0", "--quiet"])

_ensure_semopy()
_ensure_factor_analyzer()

from semopy import Model as SemopyModel
from semopy.stats import (
    get_baseline_model, calc_dof, calc_chi2, calc_cfi, calc_tli, calc_rmsea, calc_aic, calc_bic
)
from factor_analyzer import FactorAnalyzer, calculate_kmo, calculate_bartlett_sphericity

# ---------- Config ----------
IN_PARQUET = rd_join(OUT_DIR, "cdr_keep_after_attn.parquet")

ITEM_META = [
    ("CDR attributions _1",  "Environmental"),
    ("CDR attributions _2",  "Environmental"),
    ("CDR attributions _3",  "Environmental"),
    ("CDR attributions _4",  "Social"),
    ("CDR attributions _5",  "Social"),
    ("CDR attributions _6",  "Social"),
    ("CDR attributions _7",  "Social"),
    ("CDR attributions _8",  "Social"),
    ("CDR attributions _9",  "Social"),
    ("CDR attributions _10", "Social"),
    ("CDR attributions _11", "Social"),
    ("CDR attributions _12", "Social"),
    ("CDR attributions _13", "Governance"),
    ("CDR attributions _14", "Governance"),
    ("CDR attributions _15", "Governance"),
    ("CDR attributions _16", "Governance"),
    ("CDR attributions _17", "Governance"),
    ("CDR attributions _18", "Governance"),
    ("CDR attributions _19", "Governance"),
    ("CDR attributions _20", "Governance"),
]
DOMAIN_PREFIX = {"Environmental": "ENV", "Social": "SOC", "Governance": "GOV"}
EXPECTED_COUNTS = {"Environmental": 3, "Social": 9, "Governance": 8}

# ---------- Utils ----------
def md_table(df: pd.DataFrame, index: bool = False) -> str:
    try:
        return df.to_markdown(index=index)
    except Exception:
        return df.round(6).to_string(index=index)

def save_md(name, text):
    remote = rd_join(OUT_DIR, name)
    rd_write_markdown(text, remote)
    return remote

def save_csv(name, df):
    remote = rd_join(OUT_DIR, name)
    rd_write_csv_df(df, remote, index=False)
    return remote

def savefig_remote(fig, remote_name, fmt=None, dpi=200):
    import matplotlib.pyplot as plt
    fmt = fmt or remote_name.split(".")[-1].lower()
    buf = io.BytesIO()
    fig.savefig(buf, format=fmt, dpi=dpi, bbox_inches="tight")
    ct = "image/png" if fmt == "png" else "image/svg+xml" if fmt == "svg" else "application/octet-stream"
    rd_upload_bytes(rd_join(OUT_DIR, remote_name), buf.getvalue(), ct)
    buf.close()
    plt.close(fig)

def match_columns(expected, cols):
    out = {}
    norm = {c: re.sub(r"[\s_]+", "", str(c)).lower() for c in cols}
    for name in expected:
        target = re.sub(r"[\s_]+", "", name).lower()
        found = [c for c, n in norm.items() if n == target]
        if found:
            out[name] = found[0]
    return out

def fallback_match_missing(missing_expected, all_cols):
    add = {}
    all_norm = {c: re.sub(r"\s+", " ", str(c)).strip() for c in all_cols}
    for name in missing_expected:
        m = re.search(r"(\d+)$", name)
        num = int(m.group(1)) if m else None
        if num is not None:
            pattern = re.compile(rf"^cdr\s*attributions\s*[_ ]?\s*{num}\s*$", re.I)
            for col, col_norm in all_norm.items():
                if pattern.match(col_norm):
                    add[name] = col
                    break
    return add

def cronbach_alpha(df_items: pd.DataFrame) -> float:
    X = df_items.dropna()
    if X.shape[1] < 2 or X.shape[0] == 0:
        return np.nan
    k = X.shape[1]
    var_sum = X.var(axis=0, ddof=1).sum()
    total_var = X.sum(axis=1).var(ddof=1)
    if total_var == 0:
        return np.nan
    return float((k / (k - 1)) * (1 - var_sum / total_var))

def row_mean_with_min(df_items: pd.DataFrame, min_prop: float = 0.5) -> pd.Series:
    need = int(np.ceil(min_prop * df_items.shape[1])) if df_items.shape[1] else 0
    valid = df_items.notna().sum(axis=1) if df_items.shape[1] else pd.Series(0, index=df_items.index)
    m = df_items.mean(axis=1) if df_items.shape[1] else pd.Series(np.nan, index=df_items.index)
    m[valid < need] = np.nan
    return m

def _norm_cols(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    d.columns = [re.sub(r"[^a-z0-9]+", "", str(c)).lower() for c in d.columns]
    return d

def extract_std(m: SemopyModel, used_cols: list[str]):
    est = m.inspect(std_est=True)  # standardized solution
    d = _norm_cols(est)
    # identify key columns
    lhs = next(c for c in ["lhs","lval","left","from","source"] if c in d.columns)
    rhs = next(c for c in ["rhs","rval","right","to","target"] if c in d.columns)
    op  = next(c for c in ["op","operator","relation"] if c in d.columns)
    cand = ["eststd","stdest","stdestimate","standardizedestimate","std","estimate","est","value"]
    val = next((c for c in cand if c in d.columns), None)
    if val is None:
        id_cols = {lhs, rhs, op}
        num_cols = [c for c in d.columns if c not in id_cols and pd.api.types.is_numeric_dtype(d[c])]
        val = num_cols[0] if num_cols else None
        if val is None:
            return pd.DataFrame(columns=["Factor","Tagged","Std_Loading"]), pd.DataFrame(columns=["Pair","r"])

    all_names = set(d[lhs]).union(set(d[rhs]))
    fac_keys = [x for x in all_names if str(x).lower() in {"environmental","social","governance","cdr","env","sg"}]
    fac_map = {k.lower(): str(k) for k in set(map(str, fac_keys))}
    used = set(used_cols)

    # loadings: indicator ~ factor
    rows = d[(d[op] == "~") & d[rhs].str.lower().isin(fac_map.keys()) & d[lhs].isin(used)].copy()
    def _cap(name):
        nm = fac_map.get(name.lower(), name)
        if nm.lower() == "env": return "Environmental"
        if nm.lower() == "sg":  return "SocGov"
        return nm[:1].upper() + nm[1:]
    rows["Factor"] = rows[rhs].map(_cap)
    rows["Tagged"] = rows[lhs]
    rows["Std_Loading"] = pd.to_numeric(rows[val], errors="coerce")
    loads = rows[["Factor","Tagged","Std_Loading"]].dropna()

    # factor correlations
    cov_rows = d[(d[op] == "~~") & d[lhs].str.lower().isin(fac_map.keys()) & d[rhs].str.lower().isin(fac_map.keys())]
    cors = []
    for _, r in cov_rows.iterrows():
        f1 = _cap(str(r[lhs]))
        f2 = _cap(str(r[rhs]))
        if f1 < f2:
            try:
                rv = float(r[val])
            except Exception:
                rv = np.nan
            cors.append({"Pair": f"{f1}–{f2}", "r": rv})
    cor_df = pd.DataFrame(cors)
    return loads, cor_df

def fit_and_fitstats(model_str: str, data: pd.DataFrame, obj="MLW"):
    m = SemopyModel(model_str)
    m.fit(data, obj=obj)
    chi2, pval = calc_chi2(m)
    dof = calc_dof(m)
    base = get_baseline_model(m)
    try:
        base.fit(obj=m.last_result.name_obj)
    except KeyError:
        base.fit(obj="MLW")
    chi2_base, _ = calc_chi2(base)
    dof_base = calc_dof(base)
    rmsea = calc_rmsea(m, chi2, dof)
    cfi = calc_cfi(m, dof, chi2, dof_base, chi2_base)
    tli = calc_tli(m, dof, chi2, dof_base, chi2_base)
    # SRMR from residual correlations
    obs_corr = data.corr().to_numpy()
    pred_cov, _ = m.calc_sigma()
    sd = np.sqrt(np.diag(pred_cov))
    pred_corr = pred_cov / np.outer(sd, sd)
    iu = np.triu_indices_from(obs_corr)
    resid = obs_corr[iu] - pred_corr[iu]
    srmr = float(np.sqrt(np.nanmean(resid**2)))
    aic, bic = calc_aic(m), calc_bic(m)
    fit = dict(Chi2=chi2, df=dof, p=pval, CFI=cfi, TLI=tli, RMSEA=rmsea, SRMR=srmr, AIC=aic, BIC=bic, N=data.shape[0])
    return m, fit

def tagged_to_domain(tag: str) -> str:
    t = str(tag)
    if t.startswith("ENV_"): return "Environmental"
    if t.startswith("SOC_"): return "Social"
    if t.startswith("GOV_"): return "Governance"
    return ""

# ================================================
# 1) Load data + Build tagged matrix X
# ================================================
df = rd_read_parquet_df(IN_PARQUET)

# build meta table (no regex inside f-strings)
meta_rows = []
for n, d in ITEM_META:
    m = re.search(r"(\d+)$", n)
    num = int(m.group(1)) if m else None
    if num is not None:
        tag = f"{DOMAIN_PREFIX[d]}_{num:02d}"
    else:
        clean = re.sub(r"[^A-Za-z0-9]+", "", n)
        tag = f"{DOMAIN_PREFIX[d]}_{clean}"
    meta_rows.append({"Item": n, "Domain_Theory": d, "Num": num, "Tagged": tag})
meta = pd.DataFrame(meta_rows)

# tolerant column map
col_map = match_columns(meta["Item"].tolist(), df.columns)
missing = [it for it in meta["Item"] if it not in col_map]
if missing:
    col_map.update(fallback_match_missing(missing, df.columns))
present = [it for it in meta["Item"] if it in col_map]
if not present:
    raise RuntimeError("No expected CDR items found in dataframe columns.")

# numeric matrix X
X = pd.DataFrame(index=df.index)
for _, r in meta[meta["Item"].isin(present)].iterrows():
    X[r["Tagged"]] = pd.to_numeric(df[col_map[r["Item"]]], errors="coerce")

# items present per domain (avoid identification issues later)
ITEMS = {"Environmental": [], "Social": [], "Governance": []}
for c in X.columns:
    dom = tagged_to_domain(c)
    if dom:
        ITEMS[dom].append(c)

for dom in ["Environmental","Social","Governance"]:
    if len(ITEMS[dom]) < 2:
        raise RuntimeError(f"CFA identification error: '{dom}' has {len(ITEMS[dom])} indicators (need ≥2).")

ALL_COLS = ITEMS["Environmental"] + ITEMS["Social"] + ITEMS["Governance"]
data_all = X[ALL_COLS].dropna()

# ================================================
# 2) EFA (discovery, oblimin MINRES) + Adequacy + Scree + Mapping + α
# ================================================
efa_input = X.dropna()
chi2, p_bart = calculate_bartlett_sphericity(efa_input)
kmo_all, kmo_model = calculate_kmo(efa_input)
adequacy_df = pd.DataFrame([{
    "Bartlett_chi2": round(chi2, 2),
    "Bartlett_p": round(p_bart, 6),
    "KMO_overall": round(float(kmo_model), 3),
    "N": int(efa_input.shape[0]),
    "k": int(efa_input.shape[1]),
}])
save_csv("08_rq2_efa_adequacy.csv", adequacy_df)
save_md("08_rq2_efa_adequacy.md", "# RQ2 — EFA Adequacy\n\n" + md_table(adequacy_df, index=False))

# Scree
try:
    import matplotlib.pyplot as plt
    fa0 = FactorAnalyzer(rotation=None, method="minres")
    fa0.fit(efa_input)
    ev, _ = fa0.get_eigenvalues()
    fig = plt.figure(figsize=(5, 3))
    ax = fig.gca()
    xs = range(1, len(ev) + 1)
    ax.scatter(xs, ev)
    ax.plot(xs, ev)
    ax.set_title("Scree Plot — CDR Expectations (EFA)")
    ax.set_xlabel("Factor")
    ax.set_ylabel("Eigenvalue")
    ax.axhline(y=1, linestyle="--")
    fig.tight_layout()
    savefig_remote(fig, "08_rq2_efa_scree.png")
except Exception:
    pass

# EFA with 3 factors (oblimin)
efa3 = FactorAnalyzer(n_factors=3, rotation="oblimin", method="minres")
efa3.fit(efa_input)
Lmat = pd.DataFrame(efa3.loadings_, index=efa_input.columns, columns=["F1", "F2", "F3"])

# Primary assignments
assign = {f: [] for f in ["F1", "F2", "F3"]}
prim_rows = []
for item, row in Lmat.iterrows():
    best = row.abs().idxmax()
    assign[best].append(item)
    prim_rows.append({"Tagged": item, "PrimaryFactor": best, "PrimaryLoading": float(row[best])})
prim = pd.DataFrame(prim_rows)

# EFA loadings + meta
efa_loadings = Lmat.reset_index().rename(columns={"index": "Tagged"})
efa_loadings = efa_loadings.merge(meta[["Tagged","Item","Domain_Theory","Num"]], on="Tagged", how="left")
efa_loadings = efa_loadings.merge(prim, on="Tagged", how="left")
efa_loadings["abs_Load"] = efa_loadings["PrimaryLoading"].abs()
efa_loadings = efa_loadings.sort_values(["PrimaryFactor", "abs_Load"], ascending=[True, False]).drop(columns=["abs_Load"])
save_csv("08_rq2_efa_loadings_detailed.csv", efa_loadings)

# size-based label alignment to ESG
def label_factors_by_size(assignments, expected_counts):
    sizes = {k: len(v) for k, v in assignments.items()}
    remaining = set(assignments.keys())
    mapping = {}
    for target_label, target_size in sorted(expected_counts.items(), key=lambda x: -x[1]):
        best_fac = min(remaining, key=lambda fac: abs(sizes.get(fac, 0) - target_size))
        mapping[best_fac] = target_label
        remaining.remove(best_fac)
    if remaining:
        leftover_label = list(set(expected_counts.keys()) - set(mapping.values()))[0]
        mapping[remaining.pop()] = leftover_label
    return mapping

size_map = label_factors_by_size(assign, EXPECTED_COUNTS)
map_rows = []
for fac, esg in size_map.items():
    for it in assign.get(fac, []):
        map_rows.append({"Unlabeled_Factor": fac, "Assigned_ESG_Label": esg, "Tagged": it})
save_csv("08_rq2_item_mapping.csv", pd.DataFrame(map_rows))

# ---------- EFA Pretty print (FIRST) ----------
display(Markdown("# 🔎 EFA results"))
display(Markdown("## Adequacy"))
display(Markdown(md_table(adequacy_df, index=False)))

display(Markdown("## Factor → ESG mapping (size-based)"))
efa_map_df_inline = pd.DataFrame(map_rows) if map_rows else pd.DataFrame(columns=["Unlabeled_Factor","Assigned_ESG_Label","Tagged"])
display(Markdown(md_table(efa_map_df_inline, index=False) if not efa_map_df_inline.empty else "_No EFA mapping available._"))

# Cronbach’s α by discovered labels
efa_sets = {lab: [] for lab in ["Environmental","Social","Governance"]}
for fac, lab in size_map.items():
    efa_sets[lab].extend(assign.get(fac, []))

alpha_rows_efa = []
for dom in ["Environmental","Social","Governance"]:
    items = efa_sets.get(dom, [])
    frame = efa_input[items] if items else pd.DataFrame(index=efa_input.index)
    a = cronbach_alpha(frame) if frame.shape[1] >= 2 else np.nan
    alpha_rows_efa.append({"EFA_Label": dom, "k": len(items), "Cronbach_α": (round(a, 3) if pd.notna(a) else np.nan)})
alpha_efa_df = pd.DataFrame(alpha_rows_efa)
save_csv("08_rq2_efa_alpha_by_label.csv", alpha_efa_df)

display(Markdown("## Internal consistency (Cronbach’s α by discovered constructs)"))
display(Markdown(md_table(alpha_efa_df, index=False)))

display(Markdown("## Primary loadings by discovered factor"))
for f in ["F1","F2","F3"]:
    blk = efa_loadings[efa_loadings["PrimaryFactor"] == f][["Tagged","Item","Domain_Theory","PrimaryLoading"]]
    blk = blk.sort_values("PrimaryLoading", key=lambda s: s.abs(), ascending=False)
    display(Markdown(f"**{f}**"))
    display(Markdown(md_table(blk.round(3), index=False)))

# ================================================
# 3) CFA model suite (SECOND)
# ================================================
def model_3f(items):
    return "\n".join([
        f"Environmental =~ {' + '.join(items['Environmental'])}",
        f"Social        =~ {' + '.join(items['Social'])}",
        f"Governance    =~ {' + '.join(items['Governance'])}",
        "Environmental ~~ 1*Environmental",
        "Social ~~ 1*Social",
        "Governance ~~ 1*Governance",
        "Environmental ~~ Social",
        "Environmental ~~ Governance",
        "Social ~~ Governance",
    ])

def model_2nd(items):
    return "\n".join([
        f"Environmental =~ {' + '.join(items['Environmental'])}",
        f"Social        =~ {' + '.join(items['Social'])}",
        f"Governance    =~ {' + '.join(items['Governance'])}",
        "CDR =~ Environmental + Social + Governance",
        "CDR ~~ 1*CDR",
    ])

def model_2f_env_vs_socgov(items):
    env = items['Environmental']
    sg  = items['Social'] + items['Governance']
    return "\n".join([
        f"Env =~ {' + '.join(env)}",
        f"Sg  =~ {' + '.join(sg)}",
        "Env ~~ 1*Env",
        "Sg ~~ 1*Sg",
        "Env ~~ Sg",
    ])

def model_1f(items):
    all_items = items['Environmental'] + items['Social'] + items['Governance']
    return "\n".join([
        f"CDR =~ {' + '.join(all_items)}",
        "CDR ~~ 1*CDR",
    ])

MODELS = {
    "3F_first_order":    model_3f(ITEMS),
    "2ND_order":         model_2nd(ITEMS),
    "2F_Env_vs_SocGov":  model_2f_env_vs_socgov(ITEMS),
    "1F_all_items":      model_1f(ITEMS),
}

fits, models, loadings, corrs, heywood_flags = {}, {}, {}, {}, {}
for label, mdl in MODELS.items():
    m, f = fit_and_fitstats(mdl, data_all, obj="MLW")
    L, C = extract_std(m, ALL_COLS)
    L = L.merge(meta[["Tagged","Item","Domain_Theory","Num"]], on="Tagged", how="left")
    fits[label], models[label], loadings[label], corrs[label] = f, m, L, C
    heywood_flags[label] = L[L["Std_Loading"].abs() > 1]

# ---------- CFA Pretty printing (SECOND) ----------
cmp = pd.DataFrame.from_dict(fits, orient="index")[["Chi2","df","p","CFI","TLI","RMSEA","SRMR","AIC","BIC","N"]].copy()
cmp_round = cmp.copy()
for c in ["Chi2","AIC","BIC"]: cmp_round[c] = cmp_round[c].map(lambda x: round(x,2))
for c in ["CFI","TLI","RMSEA","SRMR","p"]: cmp_round[c] = cmp_round[c].map(lambda x: round(x,6) if pd.notna(x) else x)
cmp_round["df"] = cmp_round["df"].astype(int)
cmp_round["N"]  = cmp_round["N"].astype(int)

display(Markdown("# ✅ CFA model comparison (3F vs 2ND vs 2F vs 1F)"))
display(Markdown(md_table(cmp_round.reset_index().rename(columns={"index":"Model"}), index=False)))

for label in MODELS.keys():
    display(Markdown(f"### Model: `{label}`"))
    display(Markdown("**Specification**"))
    display(Markdown("```text\n" + MODELS[label] + "\n```"))
    f = pd.DataFrame([fits[label]])
    f_show = f.copy()
    for c in ["Chi2","AIC","BIC"]: f_show[c] = f_show[c].map(lambda x: round(x,2))
    for c in ["CFI","TLI","RMSEA","SRMR","p"]: f_show[c] = f_show[c].map(lambda x: round(x,6) if pd.notna(x) else x)
    f_show["df"] = f_show["df"].astype(int); f_show["N"] = f_show["N"].astype(int)
    display(Markdown("**Fit indices**"))
    display(Markdown(md_table(f_show, index=False)))

    display(Markdown("**Factor correlations (standardized)**"))
    C = corrs[label]
    if C is not None and (not C.empty):
        C2 = C.copy(); C2["r"] = C2["r"].map(lambda x: round(x,6))
        display(Markdown(md_table(C2, index=False)))
    else:
        display(Markdown("_n/a (no freely estimated factor covariances in this model)_"))

    L = loadings[label].copy()
    if "Std_Loading" in L.columns and not L.empty:
        L["Std_Loading"] = L["Std_Loading"].map(lambda x: round(x,6))
        for fac in L["Factor"].dropna().unique():
            blk = L[L["Factor"] == fac].sort_values("Std_Loading", ascending=False)[
                ["Tagged","Item","Domain_Theory","Std_Loading"]
            ]
            display(Markdown(f"**Standardized loadings — {fac}**"))
            display(Markdown(md_table(blk, index=False)))
    else:
        display(Markdown("_No loadings extracted._"))

    hey = heywood_flags[label]
    if hey is not None and not hey.empty:
        display(Markdown("> ⚠️ **Heywood check**: |standardized loading| > 1"))
        hh = hey.copy(); hh["Std_Loading"] = hh["Std_Loading"].map(lambda x: round(x,6))
        display(Markdown(md_table(hh[["Factor","Tagged","Item","Domain_Theory","Std_Loading"]], index=False)))

# ================================================
# 4) Reliability + Domain scores + Descriptives + Plot (THIRD)
# ================================================
domain_frames = {dom: data_all[ITEMS[dom]] for dom in ["Environmental","Social","Governance"]}
rel_rows = []
for dom, frame in domain_frames.items():
    rel_rows.append({"Domain": dom, "k": frame.shape[1], "Cronbach_α": round(cronbach_alpha(frame), 3) if frame.shape[1] >= 2 else np.nan})
reliability_df = pd.DataFrame(rel_rows)
save_csv("02_reliability.csv", reliability_df)
save_md("02_reliability.md", "# Reliability — CDR expectations\n\n" + md_table(reliability_df, index=False))

scores = {dom: row_mean_with_min(domain_frames[dom], min_prop=0.5) for dom in ["Environmental","Social","Governance"]}
scores_df = pd.DataFrame(scores, index=data_all.index)
scores_df["CDR_total"] = scores_df.mean(axis=1)
save_csv("08_rq2_domain_scores.csv", scores_df.reset_index(drop=True))

desc_rows = []
for col in ["Environmental","Social","Governance","CDR_total"]:
    s = scores_df[col].dropna()
    n = int(s.shape[0])
    mean = float(s.mean()) if n else np.nan
    sd = float(s.std(ddof=1)) if n > 1 else np.nan
    se = float(s.sem(ddof=1)) if n > 1 else np.nan
    ci_low = mean - 1.96 * se if n > 1 else np.nan
    ci_high = mean + 1.96 * se if n > 1 else np.nan
    desc_rows.append({"variable": col, "n": n, "mean": round(mean,3) if n else np.nan,
                      "sd": round(sd,3) if n>1 else np.nan, "se": round(se,3) if n>1 else np.nan,
                      "ci_low": round(ci_low,3) if n>1 else np.nan, "ci_high": round(ci_high,3) if n>1 else np.nan})
descriptives_df = pd.DataFrame(desc_rows)
save_csv("03_descriptives.csv", descriptives_df)
save_md("03_descriptives.md", "# Descriptives — CDR expectations (domain scores)\n\n" + md_table(descriptives_df, index=False))

try:
    import matplotlib.pyplot as plt
    m = descriptives_df.set_index("variable")["mean"].loc[["Environmental","Social","Governance","CDR_total"]]
    fig = plt.figure(figsize=(6, 4))
    ax = fig.gca()
    m.plot(kind="bar", ax=ax)
    ax.set_ylabel("Mean (1–7)")
    ax.set_title("CDR Expectations — Domain Means")
    ax.set_xticklabels(["Environmental","Social","Governance","CDR_total"], rotation=0)
    fig.tight_layout()
    savefig_remote(fig, "08_rq2_means.png")
except Exception:
    pass

# ================================================
# 5) Codebook-anchored summary (EFA + CFA + NL wording) — printed AFTER CFA (FOURTH)
# ================================================
CB_MD   = rd_join(OUT_DIR, "codebook_cdr_items.md")
EFA_DET = rd_join(OUT_DIR, "08_rq2_efa_loadings_detailed.csv")
EFA_MAP = rd_join(OUT_DIR, "08_rq2_item_mapping.csv")
OUT_CSV = rd_join(OUT_DIR, "08_rq2_codebook_anchored.csv")
OUT_MD  = rd_join(OUT_DIR, "08_rq2_codebook_anchored.md")

def read_codebook_md(remote_md_path: str) -> pd.DataFrame:
    md = rd_read_text(remote_md_path, encoding="utf-8")
    lines = [ln.strip() for ln in md.splitlines() if ln.strip().startswith("|")]
    if len(lines) < 3:
        raise RuntimeError("Codebook table not found or malformed in codebook_cdr_items.md")
    header = [c.strip() for c in lines[0].strip("|").split("|")]
    rows = []
    for ln in lines[2:]:
        cells = [c.strip() for c in ln.strip("|").split("|")]
        if len(cells) != len(header): continue
        rows.append(cells)
    df = pd.DataFrame(rows, columns=header)
    colmap = {}
    for c in df.columns:
        cl = c.lower()
        if "variabele" in cl: colmap[c] = "Item"
        elif "domein" in cl: colmap[c] = "Theory_Domain"
        elif "volledige" in cl and "vraag" in cl: colmap[c] = "Item_NL_full"
    df = df.rename(columns=colmap)
    for must in ["Item","Theory_Domain","Item_NL_full"]:
        if must not in df.columns:
            raise RuntimeError(f"Codebook missing column: {must}")
    df["Item_NL_full"] = (df["Item_NL_full"]
                          .str.replace("<br />", " ", regex=False)
                          .str.replace(r"\s+", " ", regex=True)
                          .str.strip())
    return df

codebook = read_codebook_md(CB_MD)
efa_det  = rd_read_csv_df(EFA_DET)
efa_map  = rd_read_csv_df(EFA_MAP) if True else None

# best CFA loadings from the 3F model
cfa_std = loadings["3F_first_order"].rename(columns={"Std_Loading":"Std_Loading"})
cfa_best = cfa_std.copy()
cfa_best["abs_std"] = cfa_best["Std_Loading"].abs()
cfa_best = cfa_best.sort_values(["Tagged","abs_std"], ascending=[True, False]).drop_duplicates("Tagged")
cfa_best = cfa_best.drop(columns=["abs_std"], errors="ignore")

# add numeric + tag to codebook
def _extract_num(s):
    m = re.search(r"(\d+)$", str(s))
    return int(m.group(1)) if m else np.nan
codebook["Num"] = codebook["Item"].map(_extract_num)

def _tag_from_num(n):
    if pd.isna(n): return None
    n = int(n)
    if 1 <= n <= 3:   return f"ENV_{n:02d}"
    if 4 <= n <= 12:  return f"SOC_{n:02d}"
    if 13 <= n <= 20: return f"GOV_{n:02d}"
    return None
codebook["Tagged"] = codebook["Num"].map(_tag_from_num)
codebook["Item_NL_short"] = codebook["Item_NL_full"].map(
    lambda s: re.sub(r"\s+", " ", (re.search(r"\s-\s(.+)$", s).group(1).strip() if re.search(r"\s-\s(.+)$", s) else s))
)

# factor label mapping fallback
def infer_majority_label(df_with_tagged_and_factor):
    def tag2dom(tag):
        t = str(tag)
        if t.startswith("ENV_"): return "Environmental"
        if t.startswith("SOC_"): return "Social"
        if t.startswith("GOV_"): return "Governance"
        return "Unknown"
    labels = {}
    for fac, grp in df_with_tagged_and_factor.groupby("PrimaryFactor"):
        votes = grp["Tagged"].map(tag2dom).value_counts()
        labels[fac] = votes.idxmax() if not votes.empty else "Unknown"
    return labels

if efa_map is not None and set(["Unlabeled_Factor","Assigned_ESG_Label"]).issubset(efa_map.columns):
    fac_label_map = {row["Unlabeled_Factor"]: row["Assigned_ESG_Label"] for _, row in efa_map.drop_duplicates("Unlabeled_Factor").iterrows()}
else:
    fac_label_map = infer_majority_label(efa_det)

efa_det = efa_det.copy()
efa_det["EFA_Factor_Label"] = efa_det["PrimaryFactor"].map(fac_label_map)

final = (
    codebook[["Num","Item","Tagged","Theory_Domain","Item_NL_short","Item_NL_full"]]
    .merge(efa_det[["Tagged","PrimaryFactor","PrimaryLoading","EFA_Factor_Label"]], on="Tagged", how="left")
    .merge(cfa_best[["Tagged","Std_Loading"]], on="Tagged", how="left")
)
final["Alignment"] = np.where(final["Theory_Domain"] == final["EFA_Factor_Label"], "✅ Match", "➡️ Shifted")
final = final.sort_values(["EFA_Factor_Label","PrimaryFactor","Theory_Domain","Num"]).reset_index(drop=True)

# Save + pretty print AFTER CFA
rd_write_csv_df(final, OUT_CSV, index=False)
md_show = final[["Item","Tagged","Theory_Domain","EFA_Factor_Label","PrimaryLoading","Std_Loading","Alignment","Item_NL_short"]].copy()
md_doc = [
    "# RQ2 — Codebook-anchored summary (ORIGINAL NL items)\n",
    f"_Source codebook: `{posixpath.basename(CB_MD)}`_\n\n",
    md_table(md_show.round(3), index=False),
    "\n\n> Full NL item wording is included in CSV; `Item_NL_short` keeps only the stem.\n",
]
rd_write_markdown("".join(md_doc), OUT_MD)

display(Markdown("# 📘 Codebook-anchored summary (EFA + CFA)"))
md_show_disp = md_show.copy()
md_show_disp["PrimaryLoading"] = md_show_disp["PrimaryLoading"].round(3)
md_show_disp["Std_Loading"]    = md_show_disp["Std_Loading"].round(3)
display(Markdown(md_table(md_show_disp, index=False)))

# (Optional) interactive preview
try:
    import caas_jupyter_tools
    caas_jupyter_tools.display_dataframe_to_user("08_rq2_codebook_anchored (preview)", md_show_disp)
except Exception:
    pass

# ================================================
# 6) Persist ALL outputs (CFA suite + EFA)
# ================================================
# EFA MD (includes mapping + α table)
efa_blocks = [
    "# EFA — Summary\n\n",
    "## Adequacy\n",
    md_table(adequacy_df, index=False), "\n\n",
    "## Factor → ESG mapping (size-based)\n",
    (md_table(pd.DataFrame(map_rows), index=False) if map_rows else "_No mapping._"), "\n\n",
    "## Internal consistency (Cronbach’s α by discovered constructs)\n",
    md_table(alpha_efa_df, index=False), "\n\n",
    "## Primary loadings by discovered factor\n"
]
for f in ["F1","F2","F3"]:
    blk = efa_loadings[efa_loadings["PrimaryFactor"] == f][["Tagged","Item","Domain_Theory","PrimaryLoading"]]
    blk = blk.sort_values("PrimaryLoading", key=lambda s: s.abs(), ascending=False)
    efa_blocks += [f"### {f}\n", md_table(blk.round(3), index=False), "\n\n"]
save_md("rq2_efa_summary.md", "".join(efa_blocks))

# CFA model comparison + per model details
save_csv("rq2_cfa_model_comparison.csv", cmp_round.reset_index().rename(columns={"index":"Model"}))
save_md("rq2_cfa_model_comparison.md", "# CFA model comparison\n\n" + md_table(cmp_round.reset_index().rename(columns={"index":"Model"}), index=False))
for label in MODELS.keys():
    save_csv(f"rq2_cfa_fit_{label}.csv", pd.DataFrame([fits[label]]))
    save_csv(f"rq2_cfa_loadings_{label}.csv", loadings[label])
    if corrs[label] is not None and not corrs[label].empty:
        save_csv(f"rq2_cfa_corrs_{label}.csv", corrs[label])
    parts = []
    parts.append(f"# CFA — {label}\n\n")
    parts.append("## Specification\n")
    parts.append("```text\n" + MODELS[label] + "\n```\n")
    f = pd.DataFrame([fits[label]])
    f_show = f.copy()
    for c in ["Chi2","AIC","BIC"]: f_show[c] = f_show[c].map(lambda x: round(x,2))
    for c in ["CFI","TLI","RMSEA","SRMR","p"]: f_show[c] = f_show[c].map(lambda x: round(x,6) if pd.notna(x) else x)
    f_show["df"] = f_show["df"].astype(int); f_show["N"] = f_show["N"].astype(int)
    parts.append("## Fit indices\n")
    parts.append(md_table(f_show, index=False) + "\n\n")
    parts.append("## Factor correlations (standardized)\n")
    if corrs[label] is not None and not corrs[label].empty:
        parts.append(md_table(corrs[label], index=False) + "\n\n")
    else:
        parts.append("_n/a_\n\n")
    L = loadings[label].copy()
    if not L.empty:
        for fac in L["Factor"].dropna().unique():
            blk = L[L["Factor"] == fac].sort_values("Std_Loading", ascending=False)[["Tagged","Item","Domain_Theory","Std_Loading"]]
            parts.append(f"### {fac}\n")
            parts.append(md_table(blk, index=False) + "\n\n")
    else:
        parts.append("_No loadings._\n")
    save_md(f"rq2_cfa_{label}.md", "".join(parts))

# ================================================
# 7) Notebook “done” banner + quick pointers
# ================================================
display(Markdown("> ✅ RQ2 complete — EFA → CFA → Reliability → Codebook summary"))
display(Markdown(f"- Scree: `{rd_join(OUT_DIR, '08_rq2_efa_scree.png')}`"))
display(Markdown(f"- EFA adequacy: `{rd_join(OUT_DIR, '08_rq2_efa_adequacy.csv')}`"))
display(Markdown(f"- EFA loadings (detailed): `{rd_join(OUT_DIR, '08_rq2_efa_loadings_detailed.csv')}`"))
display(Markdown(f"- EFA mapping: `{rd_join(OUT_DIR, '08_rq2_item_mapping.csv')}`"))
display(Markdown(f"- EFA α by label: `{rd_join(OUT_DIR, '08_rq2_efa_alpha_by_label.csv')}`"))
display(Markdown(f"- Codebook summary: `{OUT_CSV}` / `{OUT_MD}`"))
display(Markdown(f"- CFA comparison: `{rd_join(OUT_DIR, 'rq2_cfa_model_comparison.csv')}`"))
display(Markdown(f"- Reliability: `{rd_join(OUT_DIR, '02_reliability.csv')}`"))
display(Markdown(f"- Scores: `{rd_join(OUT_DIR, '08_rq2_domain_scores.csv')}`"))
display(Markdown(f"- Descriptives: `{rd_join(OUT_DIR, '03_descriptives.csv')}`"))
display(Markdown(f"- Means plot: `{rd_join(OUT_DIR, '08_rq2_means.png')}`"))


# 🔎 EFA results

## Adequacy

|   Bartlett_chi2 |   Bartlett_p |   KMO_overall |    N |   k |
|----------------:|-------------:|--------------:|-----:|----:|
|         33649.1 |            0 |         0.984 | 2214 |  20 |

## Factor → ESG mapping (size-based)

| Unlabeled_Factor   | Assigned_ESG_Label   | Tagged   |
|:-------------------|:---------------------|:---------|
| F1                 | Social               | SOC_05   |
| F1                 | Social               | SOC_09   |
| F1                 | Social               | SOC_11   |
| F1                 | Social               | SOC_12   |
| F1                 | Social               | GOV_13   |
| F1                 | Social               | GOV_14   |
| F1                 | Social               | GOV_15   |
| F1                 | Social               | GOV_16   |
| F1                 | Social               | GOV_17   |
| F1                 | Social               | GOV_18   |
| F1                 | Social               | GOV_19   |
| F1                 | Social               | GOV_20   |
| F2                 | Governance           | SOC_04   |
| F2                 | Governance           | SOC_06   |
| F2                 | Governance           | SOC_07   |
| F2                 | Governance           | SOC_08   |
| F2                 | Governance           | SOC_10   |
| F3                 | Environmental        | ENV_01   |
| F3                 | Environmental        | ENV_02   |
| F3                 | Environmental        | ENV_03   |

...

- CFA comparison: `ASCOR-FMG-14116-CDR (Projectfolder)/analysis_out/rq2_cfa_model_comparison.csv`

- Reliability: `ASCOR-FMG-14116-CDR (Projectfolder)/analysis_out/02_reliability.csv`

- Scores: `ASCOR-FMG-14116-CDR (Projectfolder)/analysis_out/08_rq2_domain_scores.csv`

- Descriptives: `ASCOR-FMG-14116-CDR (Projectfolder)/analysis_out/03_descriptives.csv`

- Means plot: `ASCOR-FMG-14116-CDR (Projectfolder)/analysis_out/08_rq2_means.png`

```

EFA summary: “An EFA suggested multiple dimensions with a stable Environmental cluster; however, Social and Governance items showed cross-domain primary loadings, indicating overlap between these domains.”

Next step per prereg: “Guided by theory (ESG), we tested a confirmatory 3-factor model. The CFA fit was good (CFI/TLI≈.97; RMSEA≈.05), and loadings were strong (.62–.83), supporting the intended E/S/G structure.”

Structure refinement: “Given very high interfactor correlations, we additionally specified and evaluated a second-order CDR model (and, optionally, a bifactor as sensitivity).”

```

# RQ2 — EFA CONCLUSIONS

**Goal:** Examine whether the theorised three-factor structure of *Corporate Digital Responsibility* (Environmental, Social, Governance — ESG) is supported empirically.

---

## Overall pattern
- **Environmental items (ECDR 1–3)** all loaded clearly on an *Environmental* factor → **perfect theoretical match** (✅).  
  → Focus on ecological footprint, energy use, and recycling of digital devices.
- **Social items** split into two groups:  
  - Four items (SCDR 5, 9, 11, 12) aligned with the *Social* factor (✅).  
  - Several “people-oriented responsibility” items (SCDR 4, 6, 7, 8, 10) instead loaded with Governance-type items (➡️ shifted).
- **Governance items (GCDR 13–20)** all clustered together — but this cluster aligned with the *Social* factor (➡️ shifted), suggesting that data-protection and transparency practices were perceived by respondents as part of *social* digital responsibility rather than “governance” in a narrow sense.

---

## 📊 Alignment counts
| Domain (theory) | Items in domain | ✅ Match | ➡️ Shifted | Main shift target |
|:----------------|----------------:|---------:|------------:|:-----------------|
| Environmental   | 3 | 3 | 0 | – |
| Social          | 9 | 4 | 5 | → Governance |
| Governance      | 8 | 0 | 8 | → Social |

**Overall match rate:** 7 / 20 items ≈ 35 %.

---

## Interpretive summary
- Respondents clearly distinguished *environmental* responsibility, reproducing the theoretical subscale.
- Boundaries between *social* and *governance* aspects blurred:
  - Social-type items about *well-being, inclusiveness, and fair AI* mixed with Governance-type concerns (e.g., compliance, transparency).
  - Governance-type items on *data protection and accountability* were interpreted as *social* obligations toward users and society.
- This suggests that in public perception, **data-governance and social responsibility form one combined moral domain**, rather than two separable ESG dimensions.

---

## Implication
The empirical structure implies a possible **two-factor solution**:
1. *Environmental digital responsibility*  
2. *Socio-governance responsibility* (merging social + governance ethics)

This pattern aligns with qualitative interpretations of CDR where “responsible data use” and “fair technology” are socially framed rather than institutionally governed.

---

> Exploratory analysis confirmed a distinct *Environmental* factor, while *Social* and *Governance* items loaded on a shared dimension, indicating that participants perceive corporate data and social ethics as one integrated sphere of digital responsibility.


# Measurement Summary for RQ2 (EFA → CFA) and Alignment with Preregistration


---

## CFA (Confirmatory Factor Analysis)

| Model | χ² | df | p | CFI | TLI | RMSEA | SRMR | N |
|---|---:|---:|---:|---:|---:|---:|---:|---:|
| **2ND_order** (CDR → Env, Soc, Gov) | **1070.11** | 168 | <.001 | **.973** | **.970** | **.049** | .111 | 2214 |
| 3F_first_order (Env, Soc, Gov, correlated) | 1152.92 | 170 | <.001 | .971 | .967 | .051 | **.102** | 2214 |
| 2F_Env_vs_SocGov | 1328.24 | 171 | <.001 | .966 | .962 | .055 | .112 | 2214 |
| 1F_all_items | 1634.85 | 171 | <.001 | .956 | .952 | .062 | .116 | 2214 |

### Summary
- **Best fit:** The **second-order model** (CDR → Environmental, Social, Governance) showed the best overall fit with **CFI = .973**, **TLI = .970**, and **RMSEA = .049**.  
- **Three-factor model** also fit very well (**CFI = .971**, **RMSEA = .051**) but factor correlations were very high (Environmental–Social ≈ .90; Environmental–Governance ≈ .86), indicating strong overlap.  
- **Two- and one-factor models** fit clearly worse (CFI < .97, RMSEA > .055).  

### Loadings & Heywood Cases
- Standardized loadings were very high across all models, with several exceeding 1.0 (Heywood cases).  
- These likely reflect **strong factor intercorrelations and high communalities**, not improper estimation, given the large sample size (N=2214).  
- Interpretation focuses on factor-level and domain-level constructs rather than single items.  
- For robustness, future sensitivity with ordinal estimators (e.g., WLSMV) is recommended.

---

## Alignment with Preregistration

### Preregistered Plan
- Treat **CDR_total** as the **primary outcome**, and report **domain scores** (Environmental, Social, Governance) as **secondary**.  
- Conduct EFA → CFA to verify the theoretical 3-factor ESG structure; if confirmed, test predictors via SEM (RQ3).  
- If CFA fails, use composite domain and total scores.

### Empirical Outcome
- Both **EFA and CFA** support a three-factor structure consistent with theory.  
- The **second-order model** provides the best global fit, supporting the presence of a **general CDR factor** above the three ESG domains.  
- Therefore, the preregistered decision to use **CDR_total** as the primary dependent variable is empirically justified, and domain scores remain valid secondary outcomes.

### Implementation for Analyses
1. **RQ2:**  
   - Report means and descriptives for each domain and overall CDR_total score.  
   - Note that the second-order model fits best (CFI≈.973, RMSEA≈.049).

2. **RQ3 (Predictors and Moderation):**  
   - Model predictors of CDR expectations using the second-order latent CDR factor (or composite CDR_total).  
   - For **H3a** and **H3b**, model domain-specific effects (Environmental and Social).  
   - Include **corporate trust** as moderator in both domain and higher-order models (RQ3.2).

3. **Caveats:**  
   - Report and interpret Heywood cases.  
   - Mention high factor correlations and communalities as expected for conceptually related ESG domains.

---
## EFA vs. Theory (Correction)

Exploratory factor analysis (MINRES, oblimin) recovered **three oblique factors**, but **only the Environmental cluster was clean**. 
The other two factors showed **substantial mixing between Social and Governance items** (several Governance items loading primarily with the Social cluster and vice versa). 
While the **factor sizes** were close to the expected 3 / 9 / 8, the **content alignment** of Social vs. Governance was **not cleanly replicated** in EFA (see the codebook-anchored table showing multiple “➡️ Shifted” items).

**Implication:** Because EFA did not unambiguously reproduce the intended Social vs. Governance split, we proceeded — as preregistered — to **theory-constrained CFA** to test ESG structures.

## Conclusions

*Exploratory factor analysis (oblimin rotation) supported a three-factor structure aligned with Environmental (3 items), Social (9 items), and Governance (8 items). Confirmatory factor analyses indicated excellent fit for both a correlated three-factor model and a second-order model in which a higher-order CDR factor loads on the three domains. The second-order model provided the best overall fit (χ²(168)=1070.11, CFI=.973, TLI=.970, RMSEA=.049; N=2214). Factor correlations in the three-factor model were high (Environmental–Social r≈.90; Environmental–Governance r≈.86), suggesting a strong general CDR dimension. Several standardized loadings exceeded 1.0 (Heywood cases), likely due to high communalities and factor intercorrelations. We therefore focus on latent and composite domain-level interpretations. Consistent with preregistration, we treat the overall CDR score as the primary dependent variable and domain-specific scores as secondary. For RQ3, we model both domain-specific effects (H3a, H3b) and higher-order CDR effects, including moderation by corporate trust.*

---

